# Mô hình dự đoán tương tác dựa vào thời gian và nội dung 

## Chuẩn bị dữ liệu 

In [108]:
import pandas as pd 
import numpy as np 
import sklearn
import datetime, re
import underthesea

In [109]:
gam_1_df = pd.read_csv('Data\GAM_data_sub.csv')
gam_2_df = pd.read_csv('Data\GAMeSportsVN.csv')

main_df = pd.concat([gam_2_df, gam_1_df])

main_df = main_df.dropna(subset= [ "text", "time"])
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1205 entries, 0 to 989
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   post_id                        1205 non-null   int64  
 1   text                           1205 non-null   object 
 2   post_text                      1204 non-null   object 
 3   shared_text                    70 non-null     object 
 4   original_text                  25 non-null     object 
 5   time                           1205 non-null   object 
 6   timestamp                      1195 non-null   float64
 7   image                          844 non-null    object 
 8   image_lowquality               1203 non-null   object 
 9   images                         1200 non-null   object 
 10  images_description             1200 non-null   object 
 11  images_lowquality              1205 non-null   object 
 12  images_lowquality_description  1205 non-null   object 

### Làm sạch và xử lý missing

In [110]:
main_df = main_df.loc[:, [ 'text','time', 'image', 'video','reaction_count']]
main_df.loc[main_df['reaction_count'] == 0, 'reaction_count'] = None
main_df['reaction_count'] = main_df['reaction_count'].fillna(main_df['reaction_count'].median())
main_df

,text,time,image,video,reaction_count
0,"Palette: Góc đẹp, nhìn em mở nè anh em! 😎 #GAM...",2023-11-07 19:46:57,NaN,https://scontent.fhan5-11.fna.fbcdn.net/v/t42....,226.0
1,Cái gì quan trọng thì mình nhắc lại 42 lần! 🙄 ...,2023-11-07 17:19:53,NaN,https://scontent.fhan5-8.fna.fbcdn.net/v/t42.1...,321.0
2,Tâm linh không đùa được đâu! 🫣\n\nXem thêm ở M...,2023-11-07 15:43:23,NaN,https://scontent.fhan5-8.fna.fbcdn.net/v/t42.1...,1079.0
3,Palette tung là anh em vào hứng ngay! 🥳 #GAMTI...,2023-11-06 20:44:33,NaN,https://scontent.fhan5-8.fna.fbcdn.net/v/t42.1...,970.0
4,"Khởi đầu cho các cuộc nội chiến tại CKTG, GAM ...",2023-11-06 17:58:59,NaN,https://scontent.fhan5-2.fna.fbcdn.net/v/t42.1...,987.0
...,...,...,...,...,...
985,Thời trang không phang được thời dịch 😔😔😔\nMấy...,2020-02-07 18:00:57,https://scontent.fhan5-1.fna.fbcdn.net/v/t1.64...,NaN,2092.0
986,"Bình chọn cặp đôi ""xứng đôi vừa lứa"" nhất GAM ...",2020-02-07 12:00:00,https://scontent-hkg4-2.xx.fbcdn.net/v/t1.6435...,NaN,2092.0
987,Lịch thi đấu lượt đi của GAM eSports tại #VCS ...,2020-02-05 15:56:12,https://scontent.fhan5-1.fna.fbcdn.net/v/t1.64...,NaN,2318.0
988,Nàng Long nữ Lệ Vi đã mang về chiến thắng 2-0 ...,2020-01-31 22:34:44,https://scontent-hkg4-2.xx.fbcdn.net/v/t1.6435...,NaN,2092.0


### Phân tách dữ liệu

In [111]:
main_df['time'] = pd.to_datetime(main_df['time'])
main_df['weekday'] = main_df['time'].dt.day_of_week
main_df['hour'] = main_df['time'].dt.hour
main_df.drop(['time'], inplace = True, axis = 1)

In [112]:
main_df['image'] = ~ main_df['image'].isna()
main_df['video'] = ~main_df['video'].isna()

In [113]:
main_df.head()

,text,image,video,reaction_count,weekday,hour
0,"Palette: Góc đẹp, nhìn em mở nè anh em! 😎 #GAM...",False,True,226.0,1,19
1,Cái gì quan trọng thì mình nhắc lại 42 lần! 🙄 ...,False,True,321.0,1,17
2,Tâm linh không đùa được đâu! 🫣\n\nXem thêm ở M...,False,True,1079.0,1,15
3,Palette tung là anh em vào hứng ngay! 🥳 #GAMTI...,False,True,970.0,0,20
4,"Khởi đầu cho các cuộc nội chiến tại CKTG, GAM ...",False,True,987.0,0,17


In [114]:
from underthesea import text_normalize
from underthesea import classify 

main_df['Tag'] = main_df['text'].apply(lambda x : classify(x)[0])

main_df.head()

,text,image,video,reaction_count,weekday,hour,Tag
0,"Palette: Góc đẹp, nhìn em mở nè anh em! 😎 #GAM...",False,True,226.0,1,19,doi_song
1,Cái gì quan trọng thì mình nhắc lại 42 lần! 🙄 ...,False,True,321.0,1,17,chinh_tri_xa_hoi
2,Tâm linh không đùa được đâu! 🫣\n\nXem thêm ở M...,False,True,1079.0,1,15,chinh_tri_xa_hoi
3,Palette tung là anh em vào hứng ngay! 🥳 #GAMTI...,False,True,970.0,0,20,doi_song
4,"Khởi đầu cho các cuộc nội chiến tại CKTG, GAM ...",False,True,987.0,0,17,the_thao


In [115]:
main_df.drop("text", axis= 1, inplace= True)

### Phân chia dữ liệu

In [116]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(main_df, test_size= 0.1, random_state=42)

In [117]:
len(train_df)

1084

In [118]:
len(test_df)

121

## Mã hóa dữ liệu text

In [119]:
from sklearn.preprocessing import OneHotEncoder 

tag_encoder = OneHotEncoder()
tag_df = train_df[['Tag']]
tag_encoder_1hot = tag_encoder.fit_transform(tag_df)
tag_encoder_1hot.toarray()

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [120]:
train_df[['image', 'video', 'reaction_count', 'weekday', 'hour']]

,image,video,reaction_count,weekday,hour
199,False,True,2092.0,5,21
81,True,False,2767.0,0,13
520,True,False,2092.0,6,17
626,True,False,2092.0,0,20
512,True,False,2092.0,1,13
...,...,...,...,...,...
822,True,False,2092.0,1,20
876,False,True,2092.0,0,16
912,True,False,2092.0,2,20
629,False,True,2092.0,4,20


In [121]:
train_df['reaction_count'] = train_df['reaction_count'].astype('int32')

In [122]:
train_df_label = train_df['reaction_count'].copy()
train_df = train_df.drop("reaction_count", axis = 1)

### Transfrom pipeline

In [123]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


main_att = ['image', 'video', 'weekday', 'hour']
tag_att = ['Tag']


pipe_line = Pipeline([
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), main_att),
        ('encode', OneHotEncoder(), tag_att)
    ]))
])


train_df_prepare = pipe_line.fit_transform(train_df)


In [124]:
train_df_prepare

array([[-1.55203255,  1.79385028,  0.889181  , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.64431638, -0.55746012, -1.57848978, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.64431638, -0.55746012,  1.38271516, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.64431638, -0.55746012, -0.59142147, ...,  1.        ,
         0.        ,  0.        ],
       [-1.55203255,  1.79385028,  0.39564685, ...,  0.        ,
         0.        ,  1.        ],
       [-1.55203255,  1.79385028,  0.889181  , ...,  1.        ,
         0.        ,  0.        ]])

## Training Model

### LinearRegression

In [125]:
from sklearn.linear_model import LinearRegression 

li_re = LinearRegression()
li_re.fit(train_df_prepare, train_df_label)

LinearRegression()

In [126]:
from sklearn.metrics import mean_squared_error

train_predict = li_re.predict(train_df_prepare)
li_mse = mean_squared_error(train_predict, train_df_label)
li_rmse = np.sqrt(li_mse)
li_rmse

1699.7765130027765

###  Decision Tree

In [127]:
from sklearn.tree import DecisionTreeRegressor

de_re = DecisionTreeRegressor()
de_re.fit(train_df_prepare, train_df_label)

train_predict_de = de_re.predict(train_df_prepare)
li_rmse_de = np.sqrt(mean_squared_error(train_predict_de, train_df_label))

li_rmse_de

1211.925850967673

In [128]:
from sklearn.model_selection import cross_val_score

de_score = cross_val_score(de_re, train_df_prepare, train_df_label, scoring="neg_mean_squared_error", cv=10)
li_score = cross_val_score(li_re, train_df_prepare, train_df_label, scoring="neg_mean_squared_error", cv=10)

def print_score(score) : 
    print("Mean : " + str(np.sqrt(-score).mean()) )
    print("STD: " + str(np.sqrt(-score).std()))

print_score(de_score)

print_score(li_score)

Mean : 2082.3061604164577
STD: 385.55789656945774
Mean : 1685.362677534588
STD: 367.0307548335584


## Predict

In [129]:
test_x_df = test_df.drop("reaction_count", axis = 1)
test_y_df = test_df['reaction_count'].copy()

test_x_prepare = pipe_line.transform(test_x_df)

test_predict = de_re.predict(test_x_prepare)

test_rmse = np.sqrt(mean_squared_error(test_y_df, test_predict))

test_rmse

1916.3711300502496